# **Este *ipynb* está enmarcado dentro de un trabajo que busca predecir qué alumnos del departamento de Sistemas la UTN FRBA desertarán.**

Los datos disponibles fueron unificados en una sola tabla denominada **merged_df**.

En este **ipynb** se realizará un análisis estadístico del previamente mencionado dataset.

## **Fuente de referencia:**

* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/tree/master/desercion/diagnostics/00_datos_01

* Romero, G., et al. Predictor de deserción universitaria

---

## **Importación de librerías**

In [ ]:
## Importamos librerías.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## **Google Colaboratory o Local**
El notebook podrá ser corrido tanto localmente como en Google Colaboratory.

El usuario deberá modificar el root path de acuerdo a su conveniencia.

In [ ]:
## Verificamos si estamos corriendo el noteboock en Google Colaboratory.
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)

## En el caso de estar en Google Colab, montamos nuestro Drive.
if var_google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  ## Direccion root donde está el notebook.
  root_path = "/content/gdrive/MyDrive/Colab Notebooks/GIAR/"

## En el caso de no estar en Google Colab, estamos corriendo localmente el notebook.
else:
  root_path = ""

## **Datasets**

In [ ]:
## Importamos el dataset.
baseline = pd.read_csv(root_path + 'datos/baseline_2009_completo.csv')

## **ANÁLISIS DE LOS DATOS**

In [ ]:
## Imprimimos las dimensiones del dataset.
print(f'Dimensiones del dataset: {baseline.shape}')

In [ ]:
## Imprimimos los nombres de cada columna, el tipo de dato que contiene cada una de ellas y la cantidad de no nulos presentes.
baseline.info()

In [ ]:
baseline.drop(columns='Sexo_M', inplace = True)

In [ ]:
baseline.rename(columns = {'Sexo_F':'Sexo'}, inplace = True)

In [ ]:
baseline['Sexo'].value_counts()

In [ ]:
baseline.drop(columns='Año', inplace = True)

In [ ]:
## Vemos la cantidad y % de NaN por columna.
total = baseline.isnull().sum().sort_values(ascending=False)
percent = round(baseline.isnull().sum()/baseline.isnull().count()*100,2).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent [%]'])
missing_data

In [ ]:
## Identificamos cuántos de los registros tienen las columnas 'EsTecnico' y las referidas a los grupos de cursada de materias integradoras con valores nulos.
baseline[baseline['grupo_ingreso_nivel1'].isnull()]['EsTecnico'].isnull().value_counts()

In [ ]:
## Realizamos un gráfico de torta para mostrar la distribucion de los alumnos que desertan y los que no.
proporcion = pd.DataFrame(baseline.deserto.value_counts(normalize=True))
porcentaje = list(set(proporcion['deserto']))

colors = sns.color_palette('pastel')[0:5]
plt.pie(x = porcentaje, labels = ['No desertor','Desertor'], colors = colors, autopct = '%.2f%%')
plt.show()

In [ ]:
## Diferenciamos las variables numéricas de las categóricas.
num_features = ('edad al ingreso',
                'Distancia',
                'Ciclo Lectivo de Cursada',
                'Cantidad de veces recursada regular',
                'Descripción de recursada regular_No Recurso', 
                'Descripción de recursada regular_Recurso 1 Vez', 
                'Descripción de recursada regular_Recurso 2 Veces', 
                'Descripción de recursada regular_Recurso 3 Veces', 
                'Descripción de recursada regular_Recurso 4 Veces', 
                'Descripción de recursada regular_Recurso 5 Veces', 
                'Descripción de recursada regular_Recurso n Veces (>5)', 
                'noAprobado',
                'Aprobado',
                'Promociono',
                'Nota', 
                'Nota_max_prom',                 
                'Indice_aprobacion',
                'Turno_Mañana', 
                'Turno_Noche',
                'Turno_Tarde', 
                'Tipo de aprobación_Cambio Curso',
                'Tipo de aprobación_Firmo',
                'Tipo de aprobación_Libre', 
                'Tipo de aprobación_No Firmo',
                'Tipo de aprobación_Promociono',
                'cantidad de años')

cat_features = ('EsTecnico', 'deserto', 'Sexo', 'grupo_ingreso_nivel1')

In [ ]:
## Obtenemos algunas estadísticas descriptivas de los datos disponibles.
baseline.loc[:, baseline.columns.isin(num_features)].describe().T

In [ ]:
## Determinamos la cantidad de registros con 'edad al ingreso' menor a 17 años.
baseline[baseline['edad al ingreso']<17]['edad al ingreso'].value_counts()

In [ ]:
## Vemos el porcentaje que reprecentan los registros con 'edad al ingreso' menor a 17 años respecto del total.
baseline[baseline['edad al ingreso']<17]['edad al ingreso'].count() / baseline['edad al ingreso'].count() * 100

In [ ]:
## Determinamos la cantidad de registros con 'distancia' respecto a la universidad mayor a 75 kilómetros.
baseline[baseline['Distancia']>75]['Distancia'].count()

In [ ]:
## Vemos el porcentaje que reprecentan los registros con con 'distancia' respecto a la universidad mayor a 75 kilómetros respecto del total.
baseline[baseline['Distancia']>75]['Distancia'].count() / baseline['Distancia'].count() * 100

In [ ]:
## Hay un registro con cantidad de años negativa.
baseline[baseline['cantidad de años']<0]

In [ ]:
## Evidentemente, se trata de un error. Dado que este alumno cursó ya más de 10 materias, estimaremos que se encuentra en su 2do año y modificaremos el dato.
baseline.loc[baseline[baseline['cantidad de años']<0].index,'cantidad de años'] = 2

In [ ]:
## Observamos la distribución de la variable 'Cantidad de veces recursada regular'.
sns.boxplot(y="Cantidad de veces recursada regular", data=baseline)

In [ ]:
## Aquellos valores por encima de 50 los consideramos outliers.
baseline[baseline['Cantidad de veces recursada regular']>30]['Codigo Alumno'].count()

---

**Descartamos algunos registros**

In [ ]:
baseline.shape

In [ ]:
## Elimino los registros con nulos.

## Al descartar los registros que en 'grupo_ingreso_nivel1' no tengan un valor asignado, estamos descartando necesariamente a quienes hayan 
## ingresado a la FRBA luego de haber comenzado sus estudios en otra institución.

## Por ejemplo, un alumno que asistió a la UBA y allí completó y aprobó todas las asignaturas de 1er nivel, decide cambiar de casa de estudios 
## y continuar sus estudios en la UTN. En este caso, el alumno durante su primer año en la FRBA estaría cursando materias de segundo nivel, por ende,
## no se le podría asignar un grupo_ingreso_nivel1 ya que no cursó la materia integradora de 1er nivel en esta institución.

## Solo vamos a analizar a aquellos que hayan iniciado a cursada de la carrera en la FRBA y se tenga registro de ello.
## Vale la pena mencionar que dichos alumnos representan la mayor parte de los registros disponibles.

df = baseline.dropna(how = 'any')

In [ ]:
## Aquellos registros con valores por encima de 30 los consideramos outliers. Eliminaremos dichos registros.
df[df['Cantidad de veces recursada regular']>30]['Codigo Alumno'].count()

In [ ]:
df.drop(df[df['Cantidad de veces recursada regular'] > 30].index, inplace=True)

In [ ]:
## Observamos la distribución de la variable 'Cantidad de veces recursada regular'.
sns.boxplot(y="Cantidad de veces recursada regular", data=df)

In [ ]:
df[df['Distancia']>75]['Codigo Alumno'].count()

In [ ]:
## Aquellos registros con distancia mayor a 75 km consideramos se encuentran desactualizados. 
## Es habitual que alumnos originarios de ciudades no comprendidas por el AMBA se desplacen hacia la ciudad
## durante la época de clases y habiten allí.
## Consideramos que estos alumnos habitan en CABA durante el período de clases y no actualizaron su domicilio.

df.loc[df['Distancia'] > 75, 'Distancia'] = 5.10

In [ ]:
df[df['Distancia']==5.10]['Codigo Alumno'].count()

In [ ]:
df['Descripción de recursada regular_Recurso n Veces (>5)'].value_counts()

In [ ]:
## Eliminamos esta variable ya que actualmente no nos ofrece información.
df.drop(columns=['Descripción de recursada regular_Recurso n Veces (>5)'], inplace = True)
df.shape

In [ ]:
df['grupo_ingreso_nivel1'].value_counts()

In [ ]:
## Descartaremos aquellos grupos que luego del preprosesamiento realizado hayan quedado con menos de 10 alumnos registrados.
grupos_descartar = pd.DataFrame((df['grupo_ingreso_nivel1'].value_counts() < 10))
grupos_descartar = grupos_descartar.loc[grupos_descartar['grupo_ingreso_nivel1']==True]
grupos_descartar = list(grupos_descartar.index)
grupos_descartar

In [ ]:
df['grupo_ingreso_nivel1'].replace(grupos_descartar, np.nan, inplace = True)
df.dropna(how='any',inplace=True)

In [ ]:
df['grupo_ingreso_nivel1'].value_counts()

In [ ]:
df.shape

---
**Elaboramos gráficos para visualizar la distribución de algunas variables en función de la clase (desertor o no desertor)**

In [ ]:
## Realizamos un gráfico de torta para mostrar la distribucion de los alumnos que desertan y los que no.
proporcion = pd.DataFrame(df.deserto.value_counts(normalize=True))
porcentaje = list(set(proporcion['deserto']))

colors = sns.color_palette('pastel')[0:5]
plt.pie(x = porcentaje, labels = ['No desertor','Desertor'], colors = colors, autopct = '%.2f%%')
plt.show()

In [ ]:
## Imprimimos la distribución de distancia a la que se encuentra el domicilio declarado del alumno en función de si son o no desertores.
sns.histplot(data=df,x=df['Distancia'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Distancia")
plt.title('Distribucion de Distancia',size =15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'Cantidad de veces recursada regular' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Distancia", data=df)

In [ ]:
## Calculamos la media de la distancia en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Distancia'].mean()

In [ ]:
## Imprimimos la distribución de la edad de los alumnos al ingresar a la universidad en función de si son o no desertores.
sns.histplot(data=df,x=df['edad al ingreso'],bins=20,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("edad al ingreso")
plt.title('Distribucion de Edad al Ingreso',size = 20)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'edad al ingreso' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="edad al ingreso", data=df)

In [ ]:
## Calculamos la media de la edad al ingreso en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['edad al ingreso'].mean()

In [ ]:
## Observamos la distribución de la variable 'Cantidad de veces recursada regular' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Cantidad de veces recursada regular", data=df)

In [ ]:
## Imprimimos la distribución de 'Cantidad de veces recursada regular' en función de si son o no desertores.
sns.histplot(data=df,x=df['Cantidad de veces recursada regular'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Cantidad de veces recursada regular")
plt.title('Distribucion de Cantidad de veces recursada regular',size = 15)
plt.xlim(0,40)
plt.show()

In [ ]:
## Calculamos la media de Cantidad de veces recursada regular en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Cantidad de veces recursada regular'].mean()

In [ ]:
## Observamos la distribución de la variable 'Descripción de recursada regular_No Recurso' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Descripción de recursada regular_No Recurso", data=df)

In [ ]:
## Imprimimos la distribución de 'Descripción de recursada regular_No Recurso' en función de si son o no desertores.
sns.histplot(data=df,x=df['Descripción de recursada regular_No Recurso'],bins=30,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Descripción de recursada regular_No Recurso")
plt.title('Distribucion de Descripción de recursada regular_No Recurso',size = 15)
plt.xlim(0,40)
plt.show()

In [ ]:
## Calculamos la media de Descripción de recursada regular_No Recurso en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Descripción de recursada regular_No Recurso'].mean()

In [ ]:
## Imprimimos la distribución de 'Promociono' en función de si son o no desertores.
sns.histplot(data=df,x=df['Promociono'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Promociono")
plt.title('Distribucion de Promociono',size = 15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'Promociono' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Promociono", data=df)

In [ ]:
## Calculamos la media de Promociono en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Promociono'].mean()

In [ ]:
## Imprimimos la distribución de 'Aprobado' en función de si son o no desertores.
sns.histplot(data=df,x=df['Aprobado'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Aprobado")
plt.title('Distribucion de Aprobado',size = 15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'Aprobado' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Aprobado", data=df)

In [ ]:
## Calculamos la media de Aprobado en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Aprobado'].mean()

In [ ]:
## Imprimimos la distribución de 'noAprobado' en función de si son o no desertores.
sns.histplot(data=df,x=df['noAprobado'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("noAprobado")
plt.title('Distribucion de noAprobado',size = 15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'noAprobado' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="noAprobado", data=df)

In [ ]:
## Calculamos la media de noAprobado en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['noAprobado'].mean()

In [ ]:
## Imprimimos la distribución de 'Indice_aprobacion' en función de si son o no desertores.
sns.histplot(data=df,x=df['Indice_aprobacion'],bins=10,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("Indice_aprobacion")
plt.title('Distribucion de Indice_aprobacion',size = 15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'Indice_aprobacion' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="Indice_aprobacion", data=df)

In [ ]:
## Calculamos la media de Indice_aprobacion en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['Indice_aprobacion'].mean()

In [ ]:
## Imprimimos la distribución de 'cantidad de años' en función de si son o no desertores.
sns.histplot(data=df,x=df['cantidad de años'],bins=6,kde=True,color=colors,stat='count',hue='deserto')
plt.xlabel("cantidad de años")
plt.title('Distribucion de cantidad de años',size = 15)
plt.show()

In [ ]:
## Observamos la distribución individuald de la variable 'cantidad de años' (DATAFRAME SIN NaNs)
sns.boxplot(x="deserto", y="cantidad de años", data=df)

In [ ]:
## Calculamos la media de cantidad de años en función del outcome de la variable 'deserto'.
df.groupby('deserto', as_index=False)['cantidad de años'].mean()

In [ ]:
## Proporción de mujeres en el dataset.
baseline['Sexo'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre las mujeres.
baseline[baseline['Sexo'] == 1]['deserto'].value_counts(normalize = True)

In [ ]:
## Proporción desertores entre los hombres.
baseline[baseline['Sexo'] == 0]['deserto'].value_counts(normalize = True)

---
---
---
---
---

In [ ]:
## Creamos un dataset sin 'Código Alumno'.
xy = df.loc[:, ~df.columns.isin(('Codigo Alumno',))].dropna(how='any')

In [ ]:
## Elaboramos e imprimimos la matriz de correlación lineal de Pearson entre las features.
sns.set_style("white")
sns.set_context("talk")
sns.set_style("ticks")
plt.subplots(figsize=(11, 11))
sns.heatmap(np.corrcoef(xy.T), xticklabels=xy.columns, yticklabels=xy.columns)
plt.title('Matriz de correlación lineal de Pearson')
plt.show()

##### **# Dataset Supervisado: Valores en función de la clase** ####

Como es un dataset supervisado, podría ser interesante ver los valores separados por clase. Por lo tanto, con el objeto solamente de realizar un análisis exploratorio, el dataset se divide según la clase y se realizan los estadísticos nuevamente.


In [ ]:
## Dividimos el dataset.
desertores = df[df['deserto']==1]
no_desertores = df[df['deserto']==0]

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase desertores.
desertores.loc[:, df.columns.isin(num_features)].describe().T

In [ ]:
## Obtenemos algunas estadísticas descriptivas de la clase NO desertores.
no_desertores.loc[:, df.columns.isin(num_features)].describe().T

---

---

## **Observaciones extraídas de la abajo mencionada fuente:**

* El dataset se encuentra balanceado. **43.88%** de los alumnos del dataset desertó mientras que el **56.12%** sigue en carrera.

* La variable **EsTecnico** tiene un **13.32%** de datos nulos. Dependiendo el método que se use podrá tolerarse o no. En los casos que no se pueda tolerar, se tendrá que imputar algún valor o se podrá optar por descartar la variable.

* El valor mínimo de la variable **edad_al_ingreso** es 11. Un valor muy bajo y puede tratarse de un error. Se analizaron la cantidad de casos que existen en que esta variable tiene un valor menor a 17, que es la mínima edad que podría entrar un estudiante a la universidad respetando todos los ciclos lectivos sin adelantar ninguno de las etapas de estudio anteriores, y dicho valor es de 4 observaciones. Las cuales representan una cantidad insignificante respecto del total de observaciones 4558 (0.08%). Por lo tanto al no poder verificarlo por el momento se decide dejarlo.


### **FUENTE:** 
* https://github.com/sebajarem/Analisis_desercion_en_ingenieria/tree/master/desercion/diagnostics/00_datos_01  (**GIAR 2021** - Romero, G., et al. Predictor de deserción universitaria)
---

---
## **Observaciones a partir del presente trabajo:**


* La variable **Distancia** tiene un **1.08%** de datos nulos.

* El valor máximo de la variable **Distancia** es **2369.08** km. Un valor muy alto. Son 240 los registros que **exceden los 75 km**. Estos representan un **5.32%** respecto del total de observaciones (4558).

* Dado que se cree poco probable que un alumno viva a **más de 75 km** lineales de la universidad y se desplace a ella cotidianamente (durante el período analizado no habían cursadas virtuales), se considera que esos datos **no** están actualizados. Es habitual que alumnos originarios de ciudades no comprendidas por el AMBA se desplacen hacia la capital del país durante la época de clases y habiten allí. Consideramos que estos alumnos (con **Distancia > 75**) habitaban en CABA durante el período de clases, pero no actualizaron su domicilio. Se les asignará, por ende, 5.1 km como distancia a la universidad.

* Hay un registro con **cantidad de años** negativa. Evidentemente, se trata de un error. Dado que ese alumno cursó ya más de 10 materias, estimaremos que se encuentra en su 2do año y modificaremos el dato.

* Luego de remover los registros que tenían valores nulos, podemos afirmar que el dataset sigue balanceado. **41.14%** de los alumnos del dataset desertó, mientras que el **58.86%** sigue en carrera.


* El porcentaje de desertores entre los alumnos de **sexo masculino** es del **44.58%**, mientras que entre los de **sexo femenino** dicho porcentaje desciende al **39.38%**.

---

---

In [ ]:
## Guardamos el dataframe en un csv.
df.to_csv(root_path + 'datos/base_datos_estudiantes_completo.csv',index=False)